In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to lan on moon was")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/apla-supply-chain-sustainability-senior/job/R-54169")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























APLA, Supply Chain Sustainability, Senior










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to P

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job posting and return it in JSON format containing the following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
# print(res.content)
type(res.content)

str

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'APLA, Supply Chain Sustainability, Senior',
 'experience': '5+ years professional experience in manufacturing sustainability & strategy deployment at country and factory levels',
 'skills': ['Deploying and translating sustainability strategy into actions',
  'Capability building and iterative problem solving',
  'Stakeholder and partner management skills',
  'Project Management skills and strong attention to detail',
  'Performance management',
  'Building & translating sustainability data and analytics',
  'Ability to use data to make the business case for initiatives',
  'Excellent verbal and written English & Spanish communication skills'],
 'description': 'We’re looking for a APLA, Supply Chain Sustainability, Senior (focused on Latin America) to be based in Mexico City. This role works with key apparel & materials manufacturers, and their leadership in sustainability is essential to ensure we meet our targets. To be successful in this role, we need someone with deep expe

In [6]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=["Experience in python", "Expertise in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [13]:
job = json_res
job['skills']

['Deploying and translating sustainability strategy into actions',
 'Capability building and iterative problem solving',
 'Stakeholder and partner management skills',
 'Project Management skills and strong attention to detail',
 'Performance management',
 'Building & translating sustainability data and analytics',
 'Ability to use data to make the business case for initiatives',
 'Excellent verbal and written English & Spanish communication skills']

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [15]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are a hiring manager at Abc company. Abc is an AI and software development company with seamless integration of business tools.
    Over our experience, we have numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction and heightened efficiency.
    Your job is to write a code email to the client regarding the job mentioned above by fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Abc's portfolio: {link_list}
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), 'link_list': links})
print(res.content)

Subject: Expert Solution for APLA, Supply Chain Sustainability, Senior Role

Dear [Client],

We understand that you're seeking a highly skilled professional to fill the APLA, Supply Chain Sustainability, Senior role, focused on Latin America, to be based in Mexico City. Our team at Abc is confident that we can provide the ideal candidate with the required expertise and skills to drive sustainability strategies in global supply chains.

Our proposed candidate has a strong background in manufacturing sustainability and strategy deployment at country and factory levels, with over 5 years of professional experience. They possess excellent skills in:

* Deploying and translating sustainability strategy into actions
* Capability building and iterative problem solving
* Stakeholder and partner management skills
* Project Management skills and strong attention to detail
* Performance management
* Building & translating sustainability data and analytics
* Ability to use data to make the busines